In [1]:
# Import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("test") \
      .getOrCreate()

In [2]:
def get_length_articulos(source_path, target_path):
     df_articles = spark.read.option("header",True) \
          .csv(source_path)

     df_product_type_dict = df_articles.select('product_type_no', 'product_type_name').dropDuplicates(['product_type_no']).orderBy('product_type_no')

     df_product_type_dict.coalesce(1).write.mode('overwrite').option("header",True) \
          .csv(target_path)

     df_product_type = spark.read.option("header",True) \
          .csv(target_path)

     df = df_articles.join(df_product_type, df_articles.product_type_no == df_product_type.product_type_no, "inner")

     if df_articles.count() == df.count():
          return True, df.count()

In [3]:
source_csv_path = "../data/articles.csv"
target_csv_path = "../data/product_type.csv"

same_length, count = get_length_articulos(source_csv_path, target_csv_path)
if same_length:
    print(f"There's count consistency before and after joining dataframes! - {count} rows")
else:
    print("There's a mismatch in the count of records after the join. Please validate again.")

There's count consistency before and after joining dataframes! - 105542 rows
